# Работа с таблицами (упражнения)

Применяем функции `unstack()`, `wide_to_long()`, `melt()`

## Загрузка библиотек

In [1]:
import pandas as pd
import numpy as np

## Изучаем покемонов и применяем `unstack()`

In [2]:
pokemon = pd.read_csv('./Pokemon.csv')

In [3]:
pokemon.dtypes

#              int64
Name          object
Type 1        object
Type 2        object
Total          int64
HP             int64
Attack         int64
Defense        int64
Sp. Atk        int64
Sp. Def        int64
Speed          int64
Generation     int64
Legendary       bool
dtype: object

In [4]:
print(pokemon.shape)
print(pokemon.dropna().shape[0])
print(pokemon.drop_duplicates().shape[0])

(800, 13)
414
800


Сначала изменим названия исходных столбцов:

- пробелы и точки нужно заменить на ```"_"``` (напр. ```Sp. Atk``` → ```sp_atk```).
- приведите все названия к нижнему регистру
- колонку ```"#"``` переименовать в ```"id"```

Полученные результаты запишем в исходный датафрейм ```pokemon```.

Затем сгруппируем данные по поколению покемонов (```generation```), и с помощью ```value_counts()``` посчитаем, сколько в каком поколении легендарных покемонов (```legendary```), а также сколько в этих поколениях нелегендарных покемонов. Полученный объект приведем к формату датафрейма (```.to_frame()```) и сохраним в ```legends```.

In [5]:
pokemon.columns = \
    pokemon.columns \
    .str.replace(r'\s+', '_', regex = True) \
    .str.replace('.', '', regex = False) \
    .str.lower()
pokemon = pokemon.rename(columns = {'#': 'id'})

In [6]:
pokemon.head()

,id,name,type_1,type_2,total,hp,attack,defense,sp_atk,sp_def,speed,generation,legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [7]:
pokemon \
    .groupby(['generation', 'legendary']) \
    .agg({'legendary': 'count'}) \
    .rename(columns = {'legendary': 'count'})

count
generation legendary       
1          False        160
           True           6
2          False        101
           True           5
3          False        142
           True          18
4          False        108
           True          13
5          False        150
           True          15
6          False         74
           True           8

In [8]:
legends = pokemon \
    .groupby('generation') \
    ['legendary'] \
    .value_counts() \
    .to_frame()
legends

legendary
generation legendary           
1          False            160
           True               6
2          False            101
           True               5
3          False            142
           True              18
4          False            108
           True              13
5          False            150
           True              15
6          False             74
           True               8

Видно, что есть только одна колонка со значениями, которая называется ```legendary```, а в качестве индекса используются две колонки — ```generation``` и ```legendary```. Два одинаковых названия — не очень хорошо, поэтому необходимо переименовать колонку ```legendary``` в ```legendary_count```. 

Используем таблицу ```legends```, полученную на предыдущем шаге, и изменим в нём название столбца, перезаписав его в ту же переменную. Затем используем ```unstack```, чтобы поместить уровень индекса ```legendary``` в уровень оси столбцов. Иными словами, должно получиться две колонки – ```False``` & ```True```. Результат сохраним в ```legends_unstacked```.

In [10]:
legends = legends.rename(columns = {'legendary': 'legendary_count'})
legends_unstacked = legends.unstack(level = 1)
legends_unstacked

legendary_count     
legendary            False True
generation                     
1                      160    6
2                      101    5
3                      142   18
4                      108   13
5                      150   15
6                       74    8

Теперь попробуем узнать, среди каких типов покемонов и какого поколения больше всего легендарных.

Сгруппируем таблицу ```pokemon``` по переменным ```generation``` и ```type_1```, посчитаем количество легендарных покемонов внутри групп. Приведиём данные в формат датафрейма, а затем используем ```unstack()```. В качестве ответа выберем вид и поколение покемона, среди которых больше всего легендарных.

In [39]:
legendary_pivot = \
    pokemon[pokemon.legendary] \
    .groupby(['generation', 'type_1']) \
    .agg({'legendary': 'count'})

In [40]:
legendary_pivot \
    .unstack(1) \
    .fillna(0)

legendary                                                       \
type_1          Dark Dragon Electric Fairy Fire Flying Ghost Grass Ground   
generation                                                                  
1                0.0    0.0      1.0   0.0  1.0    0.0   0.0   0.0    0.0   
2                0.0    0.0      1.0   0.0  2.0    0.0   0.0   0.0    0.0   
3                0.0    6.0      0.0   0.0  0.0    0.0   0.0   0.0    2.0   
4                1.0    0.0      0.0   0.0  1.0    0.0   2.0   2.0    0.0   
5                0.0    5.0      2.0   0.0  0.0    2.0   0.0   1.0    2.0   
6                1.0    1.0      0.0   1.0  1.0    0.0   0.0   0.0    0.0   

                                                 
type_1      Ice Normal Psychic Rock Steel Water  
generation                                       
1           1.0    0.0     3.0  0.0   0.0   0.0  
2           0.0    0.0     1.0  0.0   0.0   1.0  
3           1.0    0.0     4.0  1.0   2.0   2.0  
4           0.0    2.0     3.0  0.0   1.0   1.0  
5           0.0    0.0     1.0  1.0   1.0   0.0  
6           0.0    0.0     2.0  2.0   0.0   0.0

In [41]:
legendary_pivot.idxmax()

legendary    (3, Dragon)
dtype: object

## Изучаем авокадо и применяем `wide_to_long()`

In [42]:
avocado_agg = pd.DataFrame({'type' : ['conventional', 'organic'],
                            'AvgPrice_2015' : [1.077963, 1.673324],
                            'AvgPrice_2016' : [1.105595, 1.571684],
                            'AvgPrice_2017' : [1.294888, 1.735521],
                            'AvgPrice_2018' : [1.127886, 1.567176],
                            })
avocado_agg

,type,AvgPrice_2015,AvgPrice_2016,AvgPrice_2017,AvgPrice_2018
0,conventional,1.077963,1.105595,1.294888,1.127886
1,organic,1.673324,1.571684,1.735521,1.567176


In [47]:
avocado_agg_long = \
    pd.wide_to_long(avocado_agg,
                    stubnames = 'AvgPrice',
                    i = 'type',
                    j = 'year',
                    sep = '_')
avocado_agg_long

,,AvgPrice
type,year,
conventional,2015,1.077963
organic,2015,1.673324
conventional,2016,1.105595
organic,2016,1.571684
conventional,2017,1.294888
organic,2017,1.735521
conventional,2018,1.127886
organic,2018,1.567176


## Изучаем супергероев и применяем `melt()`

In [50]:
superheroes = pd.read_csv('./superheroes_power_matrix.csv')

In [51]:
print(superheroes.shape)
print(superheroes.dropna().shape[0])
print(superheroes.drop_duplicates().shape[0])

(667, 168)
667
667


In [52]:
superheroes.head()

,Name,Agility,Accelerated Healing,Lantern Power Ring,Dimensional Awareness,Cold Resistance,Durability,Stealth,Energy Absorption,Flight,...,Web Creation,Reality Warping,Odin Force,Symbiote Costume,Speed Force,Phoenix Force,Molecular Dissipation,Vision - Cryo,Omnipresent,Omniscient
0,3-D Man,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,A-Bomb,False,True,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,Abe Sapien,True,True,False,False,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,Abin Sur,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Abomination,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [118]:
superheroes_long = \
    superheroes \
    .melt(id_vars = 'Name') \
    .rename(columns = {'variable': 'superpower'})
superheroes_long

,Name,superpower,value
0,3-D Man,Agility,True
1,A-Bomb,Agility,False
2,Abe Sapien,Agility,True
3,Abin Sur,Agility,False
4,Abomination,Agility,False
...,...,...,...
111384,Yellowjacket II,Omniscient,False
111385,Ymir,Omniscient,False
111386,Yoda,Omniscient,False
111387,Zatanna,Omniscient,False


In [128]:
superheroes_powers = \
    superheroes_long \
    .query("value") \
    [['Name', 'superpower']] \
    .groupby('Name') \
    .superpower \
    .apply(list) \
    .to_frame() \
    .reset_index()
superheroes_powers

,Name,superpower
0,3-D Man,"[Agility, Super Strength, Stamina, Super Speed]"
1,A-Bomb,"[Accelerated Healing, Durability, Longevity, S..."
2,Abe Sapien,"[Agility, Accelerated Healing, Cold Resistance..."
3,Abin Sur,[Lantern Power Ring]
4,Abomination,"[Accelerated Healing, Intelligence, Super Stre..."
...,...,...
662,Yellowjacket II,"[Flight, Energy Blasts, Size Changing]"
663,Ymir,"[Cold Resistance, Durability, Longevity, Super..."
664,Yoda,"[Agility, Stealth, Danger Sense, Marksmanship,..."
665,Zatanna,"[Cryokinesis, Telepathy, Magic, Fire Control, ..."


Иной способ вытащить значения качеств, для которых у данного супергероя стоит значение ```True```:

In [87]:
row = 0
sh_col_names = superheroes.columns[1:]
selected_row = superheroes.iloc[row, 1:]

print(superheroes.Name[row], ":", sh_col_names[selected_row].to_list())

3-D Man : ['Agility', 'Super Strength', 'Stamina', 'Super Speed']


In [95]:
sh_number = superheroes.shape[0]
heroes = [superheroes.Name[row] for row in range(0, sh_number)]
superpowers = [sh_col_names[superheroes.iloc[row, 1:]].to_list()
               for row in range(0, sh_number)]
superheroes_powers_v2 = \
    pd.DataFrame \
    .from_dict({'Name': heroes,
                'superpower': superpowers})
superheroes_powers_v2

,Name,superpower
0,3-D Man,"[Agility, Super Strength, Stamina, Super Speed]"
1,A-Bomb,"[Accelerated Healing, Durability, Longevity, S..."
2,Abe Sapien,"[Agility, Accelerated Healing, Cold Resistance..."
3,Abin Sur,[Lantern Power Ring]
4,Abomination,"[Accelerated Healing, Intelligence, Super Stre..."
...,...,...
662,Yellowjacket II,"[Flight, Energy Blasts, Size Changing]"
663,Ymir,"[Cold Resistance, Durability, Longevity, Super..."
664,Yoda,"[Agility, Stealth, Danger Sense, Marksmanship,..."
665,Zatanna,"[Cryokinesis, Telepathy, Magic, Fire Control, ..."
